In [3]:
from transformers import GroundingDinoProcessor, GroundingDinoForObjectDetection
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
from PIL import Image
model = AutoModelForZeroShotObjectDetection.from_pretrained("IDEA-Research/grounding-dino-base")
processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-base")


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [8]:
import os
import torch

In [9]:
image_path="/storage/gpfs/data-store/projects/parking-data-ops/ws/shared/project-workspace/uic19759/BDD_control_lora/wandb/latest-run/files/media/images/validation_9064_ae741a286f2bf12e35f7.png"
output_dir="dino_out"
image = Image.open(image_path).convert("RGB")
original_width, original_height = image.size

In [7]:
text_prompt = "car" # The object to detect [3]
box_threshold = 0.35 # Confidence threshold for detection [2][4]
text_threshold = 0.25 # Confidence threshold for label matching [2][4]

In [27]:
inputs = processor(images=image, text=text_prompt, return_tensors="pt")

In [28]:
# Move tensor-type values to the device
inputs = {key: value.to(device) if isinstance(value, torch.Tensor) else value for key, value in inputs.items()}

In [29]:

# --- 5. Perform Prediction ---
# Run inference
with torch.no_grad():
    outputs = model(**inputs)


In [31]:
inputs

{'input_ids': tensor([[ 101, 2482,  102]], device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1]], device='cuda:0'),
 'pixel_values': tensor([[[[-1.2788, -1.3473, -1.3987,  ..., -0.8849, -0.8849, -0.8849],
           [-1.3130, -1.3644, -1.4329,  ..., -0.8849, -0.8849, -0.9020],
           [-1.3473, -1.3987, -1.4672,  ..., -0.8849, -0.9020, -0.9192],
           ...,
           [-2.0665, -2.0665, -2.0665,  ..., -0.6965, -0.6794, -0.6623],
           [-2.1008, -2.1008, -2.1008,  ..., -0.7993, -0.7308, -0.6794],
           [-2.1179, -2.1179, -2.1179,  ..., -0.9020, -0.7822, -0.6794]],
 
          [[-0.9328, -0.9678, -1.0028,  ..., -0.5476, -0.5826, -0.6001],
           [-0.9503, -1.0028, -1.0553,  ..., -0.5476, -0.5826, -0.6001],
           [-0.9678, -1.0378, -1.1253,  ..., -0.5651, -0.5826, -0.6001],
           ...,
           [-1.9657, -1.9832, -2.0007,  ..., -0.1275, -0.1275, -0.1275],
           [-1.9482, -1.9832, -2.0182,  ...,

In [30]:

# --- 6. Post-process Results ---
# The post_process_grounded_object_detection function directly provides
# bounding boxes in [x0, y0, x1, y1] format (top-left x, top-left y, bottom-right x, bottom-right y) [4][5]
results = processor.post_process_grounded_object_detection(
    outputs,
    inputs.input_ids,
    box_threshold=box_threshold,
    text_threshold=text_threshold,
    target_sizes=[(original_height, original_width)] # Pass original image size [4][5]
)


AttributeError: 'dict' object has no attribute 'input_ids'

In [24]:

# --- 7. Extract Coordinates and Save to JSON ---
# Get results for the first (and only) image in the batch
image_results = results[0]
car_boxes = []

# The 'labels' field contains the text prompt corresponding to the detected box [4][5]
for score, label, box in zip(image_results["scores"], image_results["labels"], image_results["boxes"]):
    if label == text_prompt: # Filter for boxes specifically labeled as "car"
        # Convert tensor coordinates to integers
        box_coords = box.tolist()
        # Format: [top_left_x, top_left_y, bottom_right_x, bottom_right_y]
        car_boxes.append({
            "score": score.item(),
            "top_left_x": int(box_coords[0]),
            "top_left_y": int(box_coords[1]),
            "bottom_right_x": int(box_coords[2]),
            "bottom_right_y": int(box_coords[3])
        })

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Generate JSON filename based on image filename
base_filename = os.path.splitext(os.path.basename(image_path))[0]
json_filename = os.path.join(output_dir, f"{base_filename}_car_boxes.json")

# Save the coordinates to the JSON file
with open(json_filename, 'w') as f:
    json.dump(car_boxes, f, indent=4)

print(f"Detected {len(car_boxes)} cars in {image_path}.")
print(f"Bounding box coordinates saved to {json_filename}")

NameError: name 'results' is not defined